# 논문 토큰 호출용 클래스 정의

In [26]:
import json, base64, requests
from Crypto.Cipher import AES

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str

# 토큰 호출 함수

In [27]:
from datetime import datetime
# 나중에 AESTestClass도 호출해야 함.

def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

# 논문 API 호출 함수

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)

# 관련 논문 검색 함수
def ARTI_search(client_id, token, query):
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "search",
        "target": "ARTI",
        "searchQuery": query,
        'curPage': 1, # 현재페이지 번호
        'rowCount': 10, # 디스플레이 건수(기본값 10, 최대값 100)

    }
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml


# 단일 논문 조회 함수
def ARTI_browse(client_id, token, cn):
    
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": cn,
        "include": "",
        "exclude": None,
    }
    
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml

def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

# 논문 가져오기

In [29]:
import os, xmltodict
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

cn = 'JAKO200411922932805'

xml = ARTI_browse(CLIENT_ID, access_token, cn)

dict_data = xmltodict.parse(xml)
with open("../data/input_data.json", "w", encoding="utf-8") as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

df = xml_to_df(xml)

print(df['Title'].iloc[0])
print(df['Abstract'].iloc[0])

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=9decd9bef62baafa4a74c1bddba3e06408fe6cf4970220b159d94a2f9a162495&version=1.0&action=browse&target=ARTI&cn=JAKO200411922932805&include=
한국인의 3차원 무릎관절 구축 및 형상 측정
It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions 

In [36]:
def transform_query(input_query):

    query = {
        "BI": input_query,  # 전체
        # "TI": None,  # 논문명
        # "AU": None,  # 저자
        # "AB": None,  # 초록
        # "KW": None,  # 키워드
        # "PB": None,  # 출판사(발행기관)
        # "SN": None,  # ISSN
        # "BN": None,  # ISBN
        # "PY": None,  # 발행년도
        # "CN": None,  # 문헌번호
        # "DI": None   # DOI
    }

    json_query = json.dumps(query, separators=(',', ':')) 

    return json_query

In [37]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

queries = ['Architectural Urbanism', 'Melbourne Seoul', 'KTA projects', 'public architecture', 'civic architecture']

df = pd.DataFrame()

for query in queries:
    xml = ARTI_search(CLIENT_ID, access_token, transform_query(query))
    tmp = xml_to_df(xml)
    tmp["query"] = query
    df = pd.concat([df, tmp], ignore_index=True)

df.to_csv('../data/search_results_article.csv', index=False, encoding='utf-8')

display(df)

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=fad1aae69417d65d3efe3c64f91b5c196ae8f18e05e463e494607bb989995fb3&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%22Architectural+Urbanism%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=fad1aae69417d65d3efe3c64f91b5c196ae8f18e05e463e494607bb989995fb3&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%22Melbourne+Seoul%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=fad1aae69417d65d3efe3c64f91b5c196ae8f18e05e463e494607bb989995fb3&version=1.0&action=search&target=ARTI&searchQuery=%7B%22BI%22%3A%22KTA+projects%22%7D&curPage=1&rowCount=10
🔎 호출 URL 미리보기: https://apigateway.kisti.re

,CN,DBCode,JournalId,Publisher,JournalName,ISSN,ISBN,VolumeId,VolNo1,VolNo2,...,FulltextFlag,AbstractFlag,PageInfo,DOI,FulltextURL,ContentURL,MobileURL,Keyword,Degree,query
0,NART130570112,JAFO,NJOU00562815,EDP Sciences,SHS web of conferences,;2261-2424;,,191,192,,...,2,1,pp.01015,https://doi.org/10.1051/shsconf/202419201015,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
1,NART93923344,JAFO,NJOU00022773,Informa UK (TaylorFrancis),The journal of architecture,1360-2365;1466-4410;,,122,23,7,...,2,0,pp.1093-1103,http://dx.doi.org/10.1080/13602365.2018.1513478,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
2,NART126905066,JAFO,NJOU00557891,Emerald (MCB UP),Archnet-ijar : international journal of archit...,2631-6862;1938-7806;,,56,17,3,...,2,1,pp.478-495,https://doi.org/10.1108/arch-10-2022-0232,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
3,NART90567733,JAFO,NJOU00395424,IOP Publishing,IOP conference series : Earth and environmenta...,1755-1307;1755-1315;,,246,99,,...,2,0,pp.011002,http://dx.doi.org/10.1088/1755-1315/99/1/011002,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
4,NART90567732,JAFO,NJOU00395424,IOP Publishing,IOP conference series : Earth and environmenta...,1755-1307;1755-1315;,,246,99,,...,2,0,pp.011001,http://dx.doi.org/10.1088/1755-1315/99/1/011001,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
5,NPAP14424286,CFFO,NPRO00401518,AIP Publishing,WORLD MULTIDISCIPLINARY CIVIL ENGINEERING-ARCH...,0094-243x;;,,1,2928,,...,2,0,pp.030009,,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
6,NART88770312,JAFO,NJOU00419212,IGI Global,International journal of agricultural and envi...,1947-3192;;,,19,7,1,...,2,1,pp.1-27,https://doi.org/10.4018/ijaeis.2016010101,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
7,JAKO200934939674845,JAKO,NJOU00398833,한국공간구조학회,한국공간구조학회논문집 = Journal of the Korean Associatio...,1598-4095;2287-7401;,,54,9,3,...,2,1,pp.83-92,,http://click.ndsl.kr/servlet/OpenAPIFullTextVi...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,시간기반 프로세스 . 표면 . 잠재성 . 방법론 . 상상력 . 메트릭스 . 네트워킹,,Architectural Urbanism
8,NART86633784,JAFO,NJOU00031635,Cambridge University Press,Ancient Mesoamerica,0956-5361;1469-1787;,,59,6,,...,2,1,pp.237-250,https://doi.org/10.1017/s0956536100002212,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism
9,NART100708362,JAFO,NJOU00566886,Common Ground Research Networks,The international journal of sustainability ed...,2325-1212;2325-1220;,,31,10,3,...,2,0,pp.1-10,http://dx.doi.org/10.18848/2325-1212/CGP/v10i3...,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,,,Architectural Urbanism


In [39]:
cleaned_df = (
    df[
        ['CN', 'Title', 'Abstract', 'Pubyear', 'Author', 'ContentURL', 'query']
    ]
    .rename(
        columns={
            'CN': 'ID',
            'Title': 'title',
            'Abstract': 'description',
            'Pubyear': 'pubyear',
            'Author': 'author',
            'ContentURL': 'URL'
        }
    )
)

cleaned_df['category'] = 'article'


display(cleaned_df)

,ID,title,description,pubyear,author,URL,query,category
0,NART130570112,Sustainable Urbanism and Architectural Design:...,<P>This academic article delves into the inter...,2024,"Yan, Yi;Li, Danyu;Qin, Ke;Kong, Yixi;Wu, Xiaof...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
1,NART93923344,Architectural type and the discourse of urbanism,,2018,"Borsi, Katharina;Finney, Tarsha;Philippou, Pav...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
2,NART126905066,Architectural pedagogy for the Anthropocene: t...,<P>Purpose<P>The aim of this article is to dev...,2023,"McEwan, Cameron",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
3,NART90567733,International Conference on Sustainability in ...,,2018,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
4,NART90567732,International Conference on Sustainability in ...,,2018,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
5,NPAP14424286,Innovative tools for implementing the smart ci...,,2023,"Izbash, Andrii;Fomenko, Oksana;Danylov, Serhiy;",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
6,NART88770312,Functional Reuse and Intensification of Rural-...,,2016,"Cattaneo, Tiziano;De Lotto, Roberto;Venco, Eli...",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
7,JAKO200934939674845,랜드스케이프 어바니즘의 실천적 주제에 의한 건축도시 공간 분석,본 논문은 랜드스케이프 어바니즘의 실천적 주제를 통해 점차 도시화되어가는 건축의 실...,2009,김민경,http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
8,NART86633784,"Urbanism, Militarism, and Architectural Design...",<▼1><P><B>Abstract</B><P>The form and organiza...,1995,"Hirth, Kenneth G.",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
9,NART100708362,Using Simulation Tools in Architectural Educat...,,2015,"Elabd, Aliaa;Hallowell, George;",http://click.ndsl.kr/servlet/OpenAPIDetailView...,Architectural Urbanism,article
